In [1]:
import json
import pandas as pd
import numpy as np
from argparse import Namespace
from collections import Counter
import pickle
import os
import urllib
import networkx as nx

## Venezia-Large

### Importing Useful Data

In [2]:
GEO_metadata = pd.read_csv('data_storage/images/GEO_metadata.csv', sep='\t', index_col='Unnamed: 0')

/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
metadata = pd.read_csv('data_storage/images/metadata.csv', sep='\t', index_col='Unnamed: 0')

In [4]:
GEO_metadata.columns

Index(['y', 'x', 'street_count', 'pr', 'bc', 'dg', 'cl', 'highway', 'geometry',
       'index', 'index.2', 'index.1', 'owner', 'owner_loc', 'title',
       'description', 'comments', 'taken', 'views', 'people', 'tags', 'lat',
       'lon', 'neighbourhood', 'url', 'candownload', 'local', 'tag_list',
       'tag_len', 'NOT', 'year', 'time', 'week', 'unique_week', 'English',
       'Italian', 'Other_Lang', 'num_sent', 'revised_text', 'text_data',
       'text_len', 'GEO_node', 'posts_num', 'week_adjust', 'adjusted_week'],
      dtype='object')

In [5]:
GEO_metadata

,y,x,street_count,pr,bc,dg,cl,highway,geometry,index,...,Italian,Other_Lang,num_sent,revised_text,text_data,text_len,GEO_node,posts_num,week_adjust,adjusted_week
0,45.435812,12.335472,1,0.000063,0.000000,0.000342,0.005654,NaN,POINT (12.3354722 45.4358123),6,...,0.0,0.0,12.0,It has title of Venice - Minelli Palace Courty...,El palacio Contarini del Bovolo (también llama...,236.0,49973409,2,0,201012
1,45.435643,12.335290,3,0.000246,0.021225,0.001026,0.005649,NaN,POINT (12.3352903 45.4356434),7,...,0.0,0.0,12.0,It has title of Venice - Minelli Palace. It ha...,El palacio Contarini del Bovolo (también llama...,234.0,49981281,2,0,201012
2,45.437798,12.336022,3,0.000126,0.000000,0.000684,0.005971,NaN,POINT (12.3360219 45.4377985),10,...,0.0,0.0,6.0,"It has tags of canalgrande, canal, bridge, pie...",Canal Grande on Venetsian pääliikenneväylä. Ku...,84.0,5128847037,26,0,202033
3,45.437500,12.334590,3,0.000245,0.001504,0.001026,0.005941,NaN,POINT (12.3345897 45.4374995),11,...,0.0,0.0,4.0,"It has tags of pier, italy, venice, gondola, c...",Corona-rajoitusten aikaan oli hyvä tilaisuus k...,80.0,4370729275,42,0,202033
4,45.437222,12.333909,3,0.000262,0.001188,0.001026,0.005937,NaN,POINT (12.333909 45.4372222),12,...,0.0,0.0,4.0,"It has tags of commercialbuildings, italy, ven...",Corona-rajoitusten aikaan oli hyvä tilaisuus k...,76.0,1840378053,48,0,202033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80958,45.444009,12.332236,1,0.000046,0.000000,0.000342,0.005174,NaN,POINT (12.3322356 45.444009),116670,...,0.0,0.0,2.0,It has title of Here the packages are delivere...,"<a href=""http://www.brusatotrasporti.it/index....",35.0,5244439847,16,0,201527
80959,45.444009,12.332236,1,0.000046,0.000000,0.000342,0.005174,NaN,POINT (12.3322356 45.444009),116671,...,0.0,0.0,2.0,It has title of Door No.342. It has tags of Ve...,. It has title of Tür No.342. It has tags of v...,25.0,5244439847,16,0,201527
80960,45.444009,12.332236,1,0.000046,0.000000,0.000342,0.005174,NaN,POINT (12.3322356 45.444009),116672,...,0.0,0.0,2.0,It has title of Ave Maria. It has tags of Vene...,. It has title of Ave Maria. It has tags of ve...,26.0,5244439847,16,0,201527
80961,45.444009,12.332236,1,0.000046,0.000000,0.000342,0.005174,NaN,POINT (12.3322356 45.444009),116673,...,0.0,0.0,2.0,It has title of Kirchenportal der Church of Sa...,. It has title of Kirchenportal der Chiesa di ...,33.0,5244439847,16,0,201527


In [6]:
GEO_nodes = pd.read_csv('data_storage/images/GEO_nodes.csv', sep='\t').set_index('osmid')
GEO_nodes

,y,x,street_count,pr,bc,dg,cl,highway,geometry
osmid,,,,,,,,,
27178184,45.438197,12.335686,4,0.000197,0.208517,0.001368,0.005999,NaN,POINT (12.3356863 45.4381974)
764403528,45.438128,12.335605,3,0.000175,0.025767,0.001026,0.005985,NaN,POINT (12.3356054 45.4381284)
1675825096,45.438233,12.335728,3,0.000186,0.048636,0.001026,0.005990,NaN,POINT (12.3357281 45.4382329)
8670969688,45.438255,12.335601,3,0.000111,0.207535,0.001026,0.005998,NaN,POINT (12.3356013 45.4382549)
5395065019,45.438053,12.335932,4,0.000236,0.196443,0.001368,0.006001,NaN,POINT (12.3359319 45.4380532)
...,...,...,...,...,...,...,...,...,...
9196476006,45.441902,12.307559,3,0.000119,0.000691,0.001026,0.004071,NaN,POINT (12.3075592 45.4419018)
9196521176,45.438670,12.326683,1,0.000077,0.000000,0.000342,0.005886,NaN,POINT (12.326683 45.4386701)
9221655954,45.467211,12.279144,1,0.000030,0.000000,0.000342,0.001992,NaN,POINT (12.2791441 45.467211)


In [7]:
def check_last_week(line):
    if line['week']>50:
        if line['taken'][5:7] == '01':
            return int(-100)
    return int(0)

In [8]:
GEO_metadata['week_adjust'] = GEO_metadata.apply(lambda x: check_last_week(x),axis=1)

In [9]:
metadata['week_adjust'] = metadata.apply(lambda x: check_last_week(x),axis=1)

In [10]:
GEO_metadata['adjusted_week'] = (GEO_metadata['unique_week'].astype(int) + GEO_metadata['week_adjust']).astype(str)

In [11]:
metadata['adjusted_week'] = (metadata['unique_week'].astype(int) + metadata['week_adjust']).astype(str)

In [12]:
metadata[metadata.week>50][['taken','unique_week','adjusted_week','week_adjust']]

,taken,unique_week,adjusted_week,week_adjust
28,2020-12-22 11:15:23,202052,202052,0
29,2020-12-23 18:13:12,202052,202052,0
30,2020-12-23 17:33:12,202052,202052,0
31,2020-12-23 17:21:30,202052,202052,0
1132,2017-01-01 00:00:00,201752,201652,-100
...,...,...,...,...
116651,2015-12-23 13:40:50,201552,201552,0
116652,2015-12-24 11:45:42,201552,201552,0
116653,2015-12-24 11:47:39,201552,201552,0
116654,2015-12-24 11:46:53,201552,201552,0


In [13]:
GEO_metadata.to_csv('data_storage/images/GEO_metadata.csv', sep='\t')

In [14]:
metadata.to_csv('data_storage/images/metadata.csv', sep='\t')

In [7]:
social_links = pd.read_csv('data_storage/images/social_links.csv', sep='\t').rename(columns={'Unnamed: 0': 'User0', 
                                                                                              'Unnamed: 1': 'User1'})

In [8]:
social_links

,User0,User1,self,friend,interest,mutual
0,148125504@N04,148125504@N04,1,1,1.000000,True
1,148125504@N04,32980384@N05,0,False,0.004769,False
2,148125504@N04,96038677@N06,0,False,0.000000,False
3,148125504@N04,88380809@N07,0,False,0.012162,False
4,148125504@N04,138486769@N02,0,False,0.033613,False
...,...,...,...,...,...,...
36929924,48150664@N07,46856873@N05,0,False,0.087719,False
36929925,48150664@N07,142199786@N05,0,False,0.000000,False
36929926,48150664@N07,132571827@N06,0,False,0.047859,False
36929927,48150664@N07,29127542@N04,0,False,0.000000,False


In [9]:
GEO_node_dist = pd.read_csv('data_storage/images/GEO_node_dist.csv', sep='\t', index_col='Unnamed: 0')

In [10]:
GEO_node_dist

,27178184,764403528,1675825096,8670969688,5395065019,27178422,223527689,1863759223,2962858842,4212293019,...,9127965093,9127993414,9128012220,9196476004,9196476007,9196476006,9196521176,9221655954,9379177140,9439079311
27178184,0.0,0.7,0.4,0.7,1.8,59.5,60.0,59.3,61.4,60.7,...,78.2,127.1,147.2,216.5,215.3,215.6,89.6,555.6,75.7,165.3
764403528,0.7,0.0,1.1,1.4,2.5,58.8,59.3,58.6,60.7,60.0,...,77.5,127.0,147.1,217.2,216.0,216.3,90.3,556.3,75.0,166.0
1675825096,0.4,1.1,0.0,1.1,2.2,59.9,60.4,59.7,61.8,61.1,...,78.6,127.5,147.6,216.8,215.6,215.9,89.9,555.9,76.1,165.6
8670969688,0.7,1.4,1.1,0.0,2.5,60.2,60.7,60.0,62.1,61.4,...,78.9,127.8,147.9,215.8,214.6,214.9,88.9,554.9,76.4,164.6
5395065019,1.8,2.5,2.2,2.5,0.0,57.7,58.2,57.5,59.6,58.9,...,76.4,125.3,145.4,218.3,217.1,217.4,91.4,557.4,73.9,167.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9196476006,216.6,217.3,216.9,215.9,218.4,269.4,269.9,269.2,271.3,270.6,...,233.2,343.7,363.8,1.8,0.6,0.0,140.4,443.4,230.7,169.9
9196521176,89.6,90.3,89.9,88.9,91.4,149.1,149.6,148.9,151.0,150.3,...,132.3,216.7,236.8,140.3,139.1,139.4,0.0,479.4,129.8,102.2
9221655954,548.6,549.3,548.9,547.9,550.4,601.4,601.9,601.2,603.3,602.6,...,565.2,675.7,695.8,447.1,445.9,446.2,472.4,0.0,562.7,501.9
9379177140,75.7,75.0,76.1,76.4,73.9,48.3,48.8,48.1,50.2,49.5,...,4.9,141.0,161.1,230.6,229.4,229.7,129.8,569.7,0.0,224.0


In [11]:
5848-3549

2299

### Construct Sparse Matrix of Features

In [3]:
import scipy

#### Spatial Relationship

In [14]:
len(GEO_metadata['GEO_node'].unique())

3571

In [8]:
GEO_nodes = GEO_metadata['GEO_node'].unique()

In [9]:
GEO_dict = {GEO_nodes[i]:i for i in range(len(GEO_nodes))}

In [92]:
len(GEO_dict)

3571

In [29]:
GEO_node_dist_s = GEO_node_dist[GEO_nodes.astype(str)].loc[GEO_nodes]

In [187]:
GEO_node_dist_sym = (np.array(GEO_node_dist_s) + np.array(GEO_node_dist_s).T)/2

In [189]:
SPA_dense =  ((20 - GEO_node_dist_sym)/20 * ((GEO_node_dist_sym<20).astype(int)))

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


In [190]:
SPA_dense

array([[ 1.   ,  0.86 , -0.   , ..., -0.   , -0.   , -0.   ],
       [ 0.86 ,  1.   , -0.   , ..., -0.   , -0.   , -0.   ],
       [-0.   , -0.   ,  1.   , ..., -0.   , -0.   , -0.   ],
       ...,
       [-0.   , -0.   , -0.   , ...,  1.   ,  0.705, -0.   ],
       [-0.   , -0.   , -0.   , ...,  0.705,  1.   , -0.   ],
       [-0.   , -0.   , -0.   , ..., -0.   , -0.   ,  1.   ]])

In [191]:
SPA_sparse = scipy.sparse.csr_matrix(SPA_dense)

In [192]:
SPA_sparse

<3571x3571 sparse matrix of type '<class 'numpy.float64'>'
	with 265053 stored elements in Compressed Sparse Row format>

In [193]:
SPA_sparse.toarray()

array([[1.   , 0.86 , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.86 , 1.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   , ..., 0.   , 0.   , 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 1.   , 0.705, 0.   ],
       [0.   , 0.   , 0.   , ..., 0.705, 1.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 1.   ]])

In [194]:
scipy.sparse.save_npz('data_storage/images/sparse_SPA_matrix.npz', SPA_sparse)

In [4]:
SPA_sparse = scipy.sparse.load_npz('data_storage/images/sparse_SPA_matrix.npz')

#### Social Relationship

In [10]:
owners = GEO_metadata['owner'].unique()

In [63]:
social_links['relationship'] = (((social_links['self'].astype(int)*2 + (social_links['friend']=='True').astype(int) + (social_links['interest']>0.05).astype(int))).astype(int))/3

In [64]:
social_links['relationship'].value_counts()

0.000000    36726908
0.333333      195264
1.000000        6077
0.666667        1680
Name: relationship, dtype: int64

In [49]:
len(owners)

6077

In [51]:
len(social_links['User0'].unique())

6077

In [52]:
len(social_links['User1'].unique())

6077

In [70]:
social_links = social_links[social_links['relationship']>0]

In [71]:
social_links

,User0,User1,self,friend,interest,mutual,relationship
0,148125504@N04,148125504@N04,1,1,1.000000,True,1.000000
12,148125504@N04,141443760@N06,0,True,0.018692,False,0.333333
60,148125504@N04,90578102@N00,0,False,0.060482,False,0.333333
71,148125504@N04,151937434@N07,0,True,0.064120,False,0.666667
73,148125504@N04,74012092@N00,0,False,0.060104,False,0.333333
...,...,...,...,...,...,...,...
36929911,48150664@N07,61500086@N03,0,False,0.081551,False,0.333333
36929912,48150664@N07,114215508@N06,0,True,0.052805,False,0.666667
36929919,48150664@N07,92727359@N04,0,False,0.081081,False,0.333333
36929924,48150664@N07,46856873@N05,0,False,0.087719,False,0.333333


In [11]:
user_dict = {owners[i]:i for i in range(len(owners))}

In [89]:
len(user_dict)

6077

In [93]:
row = np.array(social_links['User0'].apply(lambda x: user_dict[x]))
col = np.array(social_links['User1'].apply(lambda x: user_dict[x]))
data = np.array(social_links['relationship'])

In [123]:
SOC_sparse = scipy.sparse.coo_matrix((data, (row, col)), shape=(len(owners), len(owners))).tocsr()

In [137]:
SOC_sparse.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [138]:
SOC_sparse

<6077x6077 sparse matrix of type '<class 'numpy.float64'>'
	with 203021 stored elements in Compressed Sparse Row format>

In [125]:
scipy.sparse.save_npz('data_storage/images/sparse_SOC_matrix.npz', SOC_sparse)

In [5]:
SOC_sparse = scipy.sparse.load_npz('data_storage/images/sparse_SOC_matrix.npz')

#### Temporal Relationship

In [12]:
times = sorted(GEO_metadata['adjusted_week'].unique())

In [113]:
len(times)

912

In [13]:
time_dict = {times[i]:i for i in range(len(times))}

In [115]:
len(time_dict)

912

In [117]:
from scipy.sparse import diags

In [131]:
TEM_sparse = diags([0.5, 1, 0.5], [-1, 0, 1], shape=(len(times), len(times))).tocsr()

In [140]:
TEM_sparse

<912x912 sparse matrix of type '<class 'numpy.float64'>'
	with 2734 stored elements in Compressed Sparse Row format>

In [136]:
TEM_sparse.toarray()

array([[1. , 0.5, 0. , ..., 0. , 0. , 0. ],
       [0.5, 1. , 0.5, ..., 0. , 0. , 0. ],
       [0. , 0.5, 1. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 0.5, 0. ],
       [0. , 0. , 0. , ..., 0.5, 1. , 0.5],
       [0. , 0. , 0. , ..., 0. , 0.5, 1. ]])

In [133]:
scipy.sparse.save_npz('data_storage/images/sparse_TEM_matrix.npz', TEM_sparse)

In [6]:
TEM_sparse = scipy.sparse.load_npz('data_storage/images/sparse_TEM_matrix.npz')

### Constructing Sparse Matrix for One-hot Encodings

In [14]:
SPA = GEO_metadata['GEO_node'].apply(lambda x: GEO_dict[x])
SPA

0           0
1           1
2           2
3           3
4           4
         ... 
80958    3567
80959    3567
80960    3567
80961    3567
80962    3567
Name: GEO_node, Length: 80963, dtype: int64

In [15]:
SPA_enc = scipy.sparse.coo_matrix((np.ones(len(SPA)), (np.array(SPA), range(len(SPA)))), shape=(len(GEO_nodes), len(SPA))).tocsr()

In [16]:
SPA_enc

<3571x80963 sparse matrix of type '<class 'numpy.float64'>'
	with 80963 stored elements in Compressed Sparse Row format>

In [17]:
SPA_enc.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
SOC = GEO_metadata['owner'].apply(lambda x: user_dict[x])
SOC

0           0
1           0
2           1
3           1
4           1
         ... 
80958    5856
80959    5856
80960    5856
80961    5856
80962    5856
Name: owner, Length: 80963, dtype: int64

In [19]:
SOC_enc = scipy.sparse.coo_matrix((np.ones(len(SOC)), (np.array(SOC), range(len(SOC)))), shape=(len(owners), len(SOC))).tocsr()

In [20]:
SOC_enc

<6077x80963 sparse matrix of type '<class 'numpy.float64'>'
	with 80963 stored elements in Compressed Sparse Row format>

In [21]:
TEM = GEO_metadata['adjusted_week'].apply(lambda x: time_dict[x])
TEM

0        351
1        351
2        891
3        891
4        891
        ... 
80958    625
80959    625
80960    625
80961    625
80962    625
Name: adjusted_week, Length: 80963, dtype: int64

In [22]:
TEM_enc = scipy.sparse.coo_matrix((np.ones(len(TEM)), (np.array(TEM), range(len(TEM)))), shape=(len(times), len(TEM))).tocsr()

In [23]:
TEM_enc

<912x80963 sparse matrix of type '<class 'numpy.float64'>'
	with 80963 stored elements in Compressed Sparse Row format>

### Constructing Adjacent Matrices

In [24]:
A_SPA = SPA_enc.T * SPA_sparse * SPA_enc

In [25]:
A_SPA

<80963x80963 sparse matrix of type '<class 'numpy.float64'>'
	with 202173159 stored elements in Compressed Sparse Column format>

In [34]:
(202173159-80963)/2

101046098.0

In [38]:
(202173159-80963)/2/80963

1248.052789545842

In [42]:
(202173159-80963)/80963/80962

0.03083058199021373

In [201]:
A_SPA.toarray()

array([[1.  , 0.86, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.86, 1.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 1.  , 1.  , 1.  ],
       [0.  , 0.  , 0.  , ..., 1.  , 1.  , 1.  ],
       [0.  , 0.  , 0.  , ..., 1.  , 1.  , 1.  ]])

In [26]:
A_SOC = SOC_enc.T * SOC_sparse * SOC_enc

In [27]:
A_SOC

<80963x80963 sparse matrix of type '<class 'numpy.float64'>'
	with 76422265 stored elements in Compressed Sparse Column format>

In [35]:
(76422265-80963)/2

38170651.0

In [39]:
(76422265-80963)/2/80963

471.4579622790657

In [43]:
(76422265-80963)/80963/80962

0.011646401083942238

In [181]:
A_SOC.toarray()

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [28]:
A_TEM = TEM_enc.T * TEM_sparse * TEM_enc

In [29]:
A_TEM

<80963x80963 sparse matrix of type '<class 'numpy.float64'>'
	with 71135671 stored elements in Compressed Sparse Column format>

In [36]:
(71135671-80963)/2

35527354.0

In [40]:
(71135671-80963)/2/80963

438.8097526030409

In [44]:
(71135671-80963)/80963/80962

0.010839894088659888

In [182]:
A_TEM.toarray()

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [31]:
A_simp = (((A_SOC>0) + (A_TEM>0) + (A_SPA>0))>0).astype(int)

In [32]:
A_simp

<80963x80963 sparse matrix of type '<class 'numpy.int64'>'
	with 290091503 stored elements in Compressed Sparse Column format>

In [37]:
(290091503-80963)/2

145005270.0

In [41]:
(290091503-80963)/2/80963

1791.0066326593628

In [45]:
(290091503-80963)/80963/80962

0.044243142033530865

In [185]:
A_simp.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])

In [177]:
scipy.sparse.save_npz('dataset_np/Venice-XL/A_SOC.npz', A_SOC)

In [30]:
scipy.sparse.save_npz('dataset_np/Venice-XL/A_SPA.npz', A_SPA)

In [179]:
scipy.sparse.save_npz('dataset_np/Venice-XL/A_TEM.npz', A_TEM)

In [33]:
scipy.sparse.save_npz('dataset_np/Venice-XL/A_simp.npz', A_simp)